In [ ]:
!pip install together


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 463.8 kB/s eta 0:00:00


In [ ]:
import json
import re  # For cleaning topic names
from together import Together
import os
from google.colab import userdata

# Initialize Together API client
TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')
client = Together(api_key=TOGETHER_API_KEY)

# Load input questions from JSON file
input_file = "/content/cleaned_questions-Removed-Pres.json"
output_file = "updated_questions.json"

with open(input_file, "r") as file:
    new_questions = json.load(file)

trainSize = 0.4
total_size = len(new_questions)
selected_data = new_questions[:int(trainSize * total_size)]

print(f"Total size of the dataset: {total_size}")
print(f"Train on size of the dataset {float(trainSize*100)}% : size:{len(selected_data)}")

new_questions = selected_data


print(new_questions[1])
# Process each question separately
updated_questions = []

for new_q in new_questions:
    prompt = f"""
Analyze the given question and assign:
1. **One most suitable Topic** (Use only a **single word or short phrase**. Avoid sentences.)
2. **One most suitable CLO (Course Learning Outcome)**

Format your response as follows:
Topic: [Your assigned topic]
CLO: [Your assigned CLO]

---

**Question:** {new_q["Question"]}
**Type:** {new_q["Type"]}
**Difficulty:** {new_q["Difficulty"]}
""".strip()

    # Making the API request
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=100,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.0
    )

    # Extract and parse the response
    generated_text = response.choices[0].message.content.strip()
    lines = generated_text.split("\n")

    # Default values
    topic, clo = "Unknown", "Unknown"

    for line in lines:
        if line.lower().startswith("topic:"):
            topic = line.split(":", 1)[1].strip()
            # Ensure topic is concise and not a full sentence
            topic = re.sub(r"[^\w\s-]", "", topic)  # Remove unwanted characters
            topic = " ".join(topic.split()[:3])  # Keep max 3 words
        elif line.lower().startswith("clo:"):
            clo = line.split(":", 1)[1].strip()

    # Update the question dictionary
    updated_q = new_q.copy()
    updated_q["Topic"] = topic
    updated_q["CLO"] = clo

    updated_questions.append(updated_q)

# Save the updated dataset to JSON file
with open(output_file, "w") as file:
    json.dump(updated_questions, file, indent=4)

print(f"Updated questions saved to {output_file}")


Total size of the dataset: 1042
Train on size of the dataset 40.0% : size:416
{'Question': 'Write a Prolog program to solve a constraint satisfaction problem using the resolution rule. The problem is defined as follows:\nImplement a Prolog program to solve a constraint satisfaction problem involving three variables (A, B, and C) with the following constraints:1. A is greater than B.\n2. B is less than C.\n3. A is not equal to C.Note: Each task builds upon the previous one, increasing in difficulty, and requires coding-based implementation with structured sub-parts. The question and cover conceptual understanding, critical thinking, and problem-solving.', 'Type': 'code', 'Difficulty': 'medium', 'CLO': 'Understand key components in the field of artificial intelligence', 'Topic': 'Define and solve constraint satisfaction problems with practical examples.', 'chunkNumber': 88, 'text': 'lifts a proof step from ground clauses up to general LIFTING LEMMA first-order clauses. In order to prove 